In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
from pandas import DataFrame

from numpy import dstack

import copy

import os
from glob import glob

from path import Path

from numpy import mean
from numpy import std

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import SimpleRNN

from keras.layers import Bidirectional

from keras.utils.vis_utils import plot_model

from keras.callbacks import EarlyStopping

from keras.optimizers import Adam

import random

from sklearn.model_selection import train_test_split

from matplotlib import pyplot

Using TensorFlow backend.


## Load Data

In [2]:
from helpers.classes.data_loader import DataLoader

In [3]:
root_path = "/Users/allarviinamae/EduWorkspace/openpose-jupyter-data-exploration/centered-keypoints"

sample_dir_names = [n for n in os.listdir(root_path) if os.path.isdir(f"{root_path}/{n}")]

samples = DataLoader.get_samples_list(sample_dir_names, root_path)

y_labels = DataLoader.get_y_labels(sample_dir_names) # classifier labels, where 0 = backflip and 1 = flack

In [4]:
i = 5

# Print some sample dir names
a = [print(sample_dir_name) for sdn_i, sample_dir_name in enumerate(sample_dir_names) if sdn_i < i]
a = [print(y_label) for y_i, y_label in enumerate(y_labels) if y_i < i]

backflip-40-margus
flack-31-rasmus
flack-19-rasmus
flack-59-martin
backflip-66-allar
0
1
1
1
0


## Data padding

In [5]:
from helpers.classes.padder import Padder

In [6]:
padded_samples_list = Padder.get_padded_samples(samples)
categorical_y_labels = DataLoader.get_categorical_y_labels(y_labels)

padded_samples_ndarray = np.asarray(padded_samples_list)
categorical_y_labels_ndarray = np.asarray(categorical_y_labels)

y_labels_stacked shape: (1, 1, 113)
y_labels_categorical shape: (1, 1, 113, 2)
y_labels_squeezed shape (113, 2)
y_label categorical: [1.0, 0.0]
y_label categorical: [0.0, 1.0]
y_label categorical: [0.0, 1.0]
y_label categorical: [0.0, 1.0]
y_label categorical: [1.0, 0.0]


## Build model

In [7]:
def get_simple_lstm_model(lstm_units, n_outputs, n_features, n_timesteps):
    optimizer = Adam(lr=0.001)
        
    model = Sequential()
    model.add(LSTM(lstm_units, input_shape=(n_timesteps, n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(lstm_units, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    return model

def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs = 1, 15
    
    #n_steps_per_epoch = 360
    
    lstm_units = 2

    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = get_simple_lstm_model(lstm_units, n_outputs, n_features, n_timesteps)
    
    
    print(model.summary())
    plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    
    batch_size = 1
    
    es_callback = EarlyStopping(monitor='val_loss', patience=3)
    
    history = model.fit(trainX,
                        trainy,
                        epochs=epochs,
                        batch_size=batch_size,
                        verbose=verbose,
                        validation_split=0.33,
                        callbacks=[es_callback])
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    
    # fit network
    #model.fit_generator(train_generator(trainX, trainy), steps_per_epoch=n_steps_per_epoch, epochs=epochs, verbose=verbose)
    # evaluate model
    #_, accuracy = model.evaluate_generator(train_generator(testX, testy), steps=len(testX), verbose=0)
    
    return history, accuracy, model

In [8]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [9]:
def save_model(model, repeat):
    # serialize model to JSON
    model_json = model.to_json()
    with open(f"model-{repeat}.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(f"model-{repeat}.h5")
    print("Saved model to disk")

## Run experiment

In [10]:
X_train, X_test, y_train, y_test = train_test_split(padded_samples_ndarray,
                                                    categorical_y_labels_ndarray,
                                                    test_size=0.2,
                                                    random_state=42)

print(f"X train len: {len(X_train)}, y train len:{len(y_train)} --- X test len:{len(X_test)}, y test len:{len(y_test)}")

train_loss_history = DataFrame()
val_loss_history = DataFrame()

train_accuracy_history = DataFrame()
val_accuracy_history = DataFrame()
# run an experiment
def run_experiment(repeats=5):
    # repeat experiment
    scores = list()
    last_history = None
    
    for r in range(repeats):
        history, score, model = evaluate_model(X_train, y_train, X_test, y_test)
        #model, score = evaluate_model(UtrainX, Utrainy, UtestX, Utesty)
        save_model(model, r)
        
        # story history
        train_loss_history[str(r)] = history.history['loss']
        val_loss_history[str(r)] = history.history['val_loss']
        train_accuracy_history[str(r)] = history.history['accuracy']
        val_accuracy_history[str(r)] = history.history['val_accuracy']
        
        score = score * 100.0
        print('>#%d validation accuracy: %.3f' % (r+1, score))
        scores.append(score)
        
    # summarize results
    summarize_results(scores)

X train len: 90, y train len:90 --- X test len:23, y test len:23


In [ ]:
run_experiment()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 2)                 264       
_________________________________________________________________
dropout_1 (Dropout)          (None, 2)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 6         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 6         
Total params: 276
Trainable params: 276
Non-trainable params: 0
_________________________________________________________________
None
Train on 60 samples, validate on 30 samples
Epoch 1/15
60/60 [==============================] - 4s 64ms/step - loss: 0.6939 - accuracy: 0.5167 - val_loss: 0.6895 - val_accuracy: 0.7000
Epoch 2/15
60/60 [==============================] - 2s 41ms/step

60/60 [==============================] - 2s 40ms/step - loss: 0.4981 - accuracy: 0.8500 - val_loss: 0.5385 - val_accuracy: 0.9333
Epoch 9/15
60/60 [==============================] - 2s 40ms/step - loss: 0.4819 - accuracy: 0.8333 - val_loss: 0.5111 - val_accuracy: 0.9333
Epoch 10/15
60/60 [==============================] - 2s 35ms/step - loss: 0.4832 - accuracy: 0.8167 - val_loss: 0.4788 - val_accuracy: 0.9333
Epoch 11/15
60/60 [==============================] - 2s 35ms/step - loss: 0.4010 - accuracy: 0.8333 - val_loss: 0.4346 - val_accuracy: 0.9333
Epoch 12/15
60/60 [==============================] - 2s 34ms/step - loss: 0.3380 - accuracy: 0.9333 - val_loss: 0.3809 - val_accuracy: 0.9667
Epoch 13/15
60/60 [==============================] - 2s 35ms/step - loss: 0.3669 - accuracy: 0.8500 - val_loss: 0.3509 - val_accuracy: 0.9667
Epoch 14/15
60/60 [==============================] - 2s 35ms/step - loss: 0.3454 - accuracy: 0.8167 - val_loss: 0.3217 - val_accuracy: 0.9333
Epoch 15/15
60/60 [

## Plots

In [ ]:
pyplot.plot(train_loss_history['0'], color='blue', label='train')
pyplot.plot(val_loss_history['0'], color='orange', label='validation')
pyplot.title('LSTM model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
#pyplot.show()

pyplot.savefig('model-train-vs-validation-loss.png')

In [ ]:
pyplot.plot(train_accuracy_history['0'], color='blue', label='train')
pyplot.plot(val_accuracy_history['0'], color='orange', label='validation')
pyplot.title('LSTM model train vs validation accuracy')
pyplot.ylabel('accuracy')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
#pyplot.show()

pyplot.savefig('model-train-vs-validation-accuracy.png')